# Prompting

In this notebook, we will explore the concept of prompting in the context of LLMs. We will discuss the different types of prompts, how they work, and how they can be used to enhance the performance of LLMs.

## Setting up the LLM using Ollama

First, we need to install the Ollama software and set up the environment. You can find the installation instructions [here](https://ollama.ai/docs/installation).

We will be using the `llama3.2:3b` model, which is a 32-bit version of the Llama model. This model is a good choice for beginners because it is lightweight and requires less computational resources.

after verifying the installation of ollama, pull the model using the following command in the terminal:

```bash
ollama pull llama3.2:3b
```

you can verify the model is downloaded by running the following command:

```bash
ollama list
```

you can test run the model by running the following command:

```bash
ollama run llama3.2:3b
```


With Ollama setup on our local machine, we can create interact with the model in python using the ollama library.

In [3]:
from langchain_ollama import ChatOllama
model_name = "llama3.2:3b"
llm = ChatOllama(model=model_name)

prompt = "What is the capital of France?"
response = llm.invoke(prompt)
print(response.content)


The capital of France is Paris.


Using `PromptTemplate` from `langchain_core.prompts` we are able to create a prompt template that can be used to generate a prompt for a given input.

In [4]:
from langchain_core.prompts import PromptTemplate

template = """
Tell me a {adjective} joke about {subject}
"""

prompt = PromptTemplate.from_template(template)

formatted_prompt = prompt.format(adjective="funny", subject="cats")
response = llm.invoke(formatted_prompt)
print(response.content)

Why did the cat join a band?

Because it wanted to be the purr-cussionist! (get it?)


# Chain of Thought Prompting

chain of thought prompting is a technique that allows the LLM to generate a response by breaking down a problem into multiple steps and generating a response for each step. This technique is useful when the LLM needs to reason about a complex problem and generate a sequence of actions to solve it.

This can be helpful when debugging exactly how the LLM is generating a response, or when the LLM is struggling to understand a complex problem.


In [5]:
cot_template = """
Consider the problem: {problem}

break down each step of your calculations
"""
cot_prompt = PromptTemplate.from_template(cot_template)

cot_formatted_prompt = cot_prompt.format(
    problem="A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. How many apples are there now?"
)
response = llm.invoke(cot_formatted_prompt)
print(response.content)

To find out how many apples are in the store now, we need to follow these steps:

Step 1: Calculate the initial number of apples after selling some:
The store had 22 apples initially.
They sold 15 apples today.

Subtracting the number of apples sold from the initial amount:
22 - 15 = 7

So, there are 7 apples left in the store before the new delivery.

Step 2: Add the new delivery to the remaining apples:
The store received a new delivery of 8 apples.
Adding these new apples to the 7 apples already present:
7 + 8 = 15

Therefore, after the sale and the new delivery, there are now 15 apples in the store.


# Few Shot Prompting
Few shot prompting is a technique where we provide a few examples of the desired output and the model generates the rest of the output. This is useful when we want the model to generate a specific output for a given input.

In [ ]:
prompt = """
Here are few examples of classifying emotions in statements:

    Statement: 'I just won my first marathon!'
    Emotion: Joy
    
    Statement: 'I can't believe I lost my keys again.'
    Emotion: Frustration
    
    Statement: 'My best friend is moving to another country.'
    Emotion: Sadness
    
    Now, classify the emotion in the following statement:
    Statement: {statement}
    Emotion:
"""
fs_prompt = PromptTemplate.from_template(prompt)
formatted_fs_prompt = fs_prompt.format(statement="I am so excited about my new job!")
response = llm.invoke(formatted_fs_prompt)
print(response.content)

Based on the statement "I am so excited about my new job!", the emotion can be classified as:

Emotion: Euphoria

Note that euphoria is a strong feeling of intense happiness or excitement, which aligns with the tone and language used in the statement.


## Formatting outputs 
from the execution above, we can notice that the output is a longer string than we what expect. In that specific case we only wanted the emotion, so we can create a custom class that the llm will have to format its output to be in the format we want.

In [7]:
from pydantic import BaseModel, Field

class EmotionClassication(BaseModel):
    emotion: str = Field(..., description="The emotion that the statement is associated with")
    
structured_llm = llm.with_structured_output(EmotionClassication)
response = structured_llm.invoke(formatted_fs_prompt)
print(response.emotion)

Excitement


# Self-consistency Prompting
self consistency prompting is a technique that allows the LLM to generate a response by comparing the generated response to a given input and generating a new response that is similar to the original response. This technique is useful when the LLM needs to generate a response that is consistent with the input.

In [8]:
sc_template = """
    {question}
    
    Provide three independent calculations and explanations, then determine the most consistent result.
"""
sc_prompt = PromptTemplate.from_template(sc_template)
formatted_sc_prompt = sc_prompt.format(question="When I was 6, my sister was half of my age. Now I am 70, what age is my sister?")
response = llm.invoke(formatted_sc_prompt)
print(response.content)


Let's calculate your sister's age using different methods.

**Method 1: Using the initial condition**

When you were 6 years old, your sister was half of your age. So, when you were 6, your sister's age was:

6 / 2 = 3

Now, let's add the number of years that have passed since then to find your sister's current age.

You are now 70 years old, so let's calculate how many years have passed since you were 6:

70 - 6 = 64 years

Since your sister was 3 years old when you were 6, her current age would be:

3 + 64 = 67 years old (for method 1)

**Method 2: Using the ratio**

When you were 6, your sister's age was half of yours. This means that for every year you age, your sister ages by half a year.

Since you are now 70 years old, we can calculate how much you aged since then:

70 - 6 = 64 years

Now, let's multiply the number of years you aged by half a year to find your sister's additional aging during that time:

64 x 0.5 = 32 years

Since your sister was 3 years old when you were 6, we 

# LCEL Chain. Piping the output of the LLM to another process.
LCEL is a technique that allows the LLM to generate a response by breaking down a problem into multiple predefined steps and generating a response for each step. This technique is useful when you want to break down a complex problem into smaller, more manageable steps and generate a response for each step or if you need to format/process the output in a specific way.

To start, lets create a simple chain that will take in a prompt, pass it to the LLM, and then return the output using `StrOutputParser`.

`RunnableLambda` is a `Runnable` that will take our input and pass it through to the next step.

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
content = """
Robotics is shifting from basic automation toward intelligent, autonomous systems that can think and act in dynamic environments rather than follow simple pre-programmed instructions. 
This trend combines analytical AI and generative AI so robots can adapt in real time to new situations.
This “Physical AI” approach builds robots that can learn from experience, generalize to different tasks and operate without strict human control.
AI integration also drives natural language and vision-based interaction, letting people command robots using more intuitive inputs.
"""

prompt = PromptTemplate(
    template="""
Use the following context to answer the question.

Context: {context}

Question: {question}

Answer:""",
    input_variables=["context", "question"],
)

chain = (
    RunnableLambda(lambda x: x)
    | prompt 
    | llm
    | StrOutputParser()
)

answer = chain.invoke({"context": content, "question": "What are some ways technology has impacted society?"})
print(answer)


Technology has had a profound impact on society in various ways. Some of the key effects include:

1. Increased efficiency and productivity: Automation and robotics have improved manufacturing processes, allowing for faster production and higher quality goods.
2. Job displacement and creation: While automation has replaced some jobs, it has also created new ones in fields such as AI development, maintenance, and management.
3. Improved healthcare: Advanced medical technologies have enabled early disease detection, personalized medicine, and more effective treatments.
4. Enhanced education: Online learning platforms and digital tools have democratized access to education, making it possible for people worldwide to acquire new skills and knowledge.
5. Changes in communication: Technology has revolutionized the way we communicate, with social media, messaging apps, and video conferencing enabling global connectivity and real-time interaction.

In the context of robotics and AI, technology

# Having multiple Prompts
Now that we have a basic understanding of how Prompts work, let's chain multiple prompts and llm responses together to create a more complex chain, and have a class to hold the information.

This example we are going to create a review chain that will take in a product review and generate a response based on the sentiment of the review. We will also be able to summarize the review in one sentence.

`RunnablePassthrough.assign` will take our input and expand on it to include the summary and sentiment of the review.

In [14]:
from langchain_core.runnables import RunnablePassthrough
import json


class Review(BaseModel):
    author: str = Field(description="The author of the review")
    product: str = Field(description="The product being reviewed")
    review: str = Field(description="The review text")
    summary: str = Field(description="The summary of the review")
    sentiment: str = Field(description="The sentiment of the review")
    response: str = Field(description="The response to the review")

    @staticmethod
    def build_review_object(data):
        return Review(
            author=data["author"],
            product=data["product"],
            review=data["review"],
            summary=data["summary"],
            sentiment=data["sentiment"],
            response=data["response"],
        )


class SentimentClassication(BaseModel):
    sentiment: str = Field(description="The sentiment that the statement is associated with")


sentiment_llm = llm.with_structured_output(SentimentClassication)

summarize_prompt = PromptTemplate.from_template(
    "Summarize this {product} review in one sentence:\n\n{review}"
)

sentiment_prompt = PromptTemplate.from_template(
    "Classify the sentiment of this review as Positive, Negative, or Neutral:\n\n{review}"
)
response_prompt = PromptTemplate.from_template(
    """Based on this product review summary and sentiment, write a professional customer service response.

Author: {author}
Product: {product}
Summary: {summary}
Sentiment: {sentiment}

Response:
"""
)


review_chain = (
    RunnablePassthrough.assign(
        summary=lambda x: (summarize_prompt | llm | StrOutputParser()).invoke(x),
        sentiment=lambda x: (sentiment_prompt | sentiment_llm).invoke(x).sentiment,
    )
    | RunnablePassthrough.assign(response=response_prompt | llm | StrOutputParser())
    | RunnableLambda(Review.build_review_object)
)


review = review_chain.invoke(
    {
        "author": "Arthur",
        "product": "Airfrier",
        "review": "This product is amazing! I've been using it for a few weeks now and I can't get enough of it. It's so easy to use and the customer service is great. I highly recommend it to anyone looking for a reliable and affordable solution.",
    }
)
print(json.dumps(dict(review), indent=2))

{
  "author": "Arthur",
  "product": "Airfrier",
  "review": "This product is amazing! I've been using it for a few weeks now and I can't get enough of it. It's so easy to use and the customer service is great. I highly recommend it to anyone looking for a reliable and affordable solution.",
  "summary": "The reviewer highly recommends the Airfrier, praising its ease of use, reliability, and affordability, as well as excellent customer service.",
  "sentiment": "Positive",
  "response": "Here's a professional customer service response:\n\nDear Arthur,\n\nThank you for taking the time to share your wonderful experience with our AirFryer product! We are thrilled to hear that it has exceeded your expectations in terms of ease of use, reliability, and affordability. Your satisfaction is of utmost importance to us, and we're delighted to know that our customer service team has been able to provide you with the support you needed.\n\nWe appreciate your kind words about our customer service, 